# Name: Nhat Minh Dang
# ID: 222172836
# SIT 378 

# Impact of Low Quality Audio: Capacity Test

## 1) Imported library:
These are the library required to process the data:

In [3]:
import os
import glob
import librosa
import numpy as np
import shutil
import wave
import pandas as pd

## 2) Path adjustment:
Set up your desired directory for originial data and newly modified data with each of the methods.

In [5]:
audio_directory_path = "Sound/Aslan"
reduce_bit_depth_directory = "Sound/Aslan_BitDepth"
stereo_to_mono_directory = "Sound/Aslan_S2M"
trim_silence_directory = "Sound/Aslan_TS"
combined_directory = "Sound/Aslan_combined"

## 3) Perform capacity measurement:
The data is first checked with origin capacity for standard. Then, the functions will be applied to modify the data with each of the method Bit Depth Reduction, Convert Stereo to Mono (S2T), Trim Silence (TS).

In [7]:
def get_directory_size(directory):
    """Get the total size of the directory in bytes."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

def reduce_bit_depth(file_path, bit_depth):
    """Reduce the bit depth of the audio file."""
    audio, sr = librosa.load(file_path, sr=None)
    max_val = 2 ** (bit_depth - 1) - 1
    reduced_audio = np.round(audio * max_val) / max_val
    reduced_audio = np.clip(reduced_audio, -1.0, 1.0)
    return reduced_audio, sr

def convert_to_mono(file_path):
    """Convert stereo audio to mono."""
    audio, sr = librosa.load(file_path, sr=None, mono=True)
    return audio, sr

def trim_silence(file_path, top_db=20):
    """Trim silence from the audio file."""
    audio, sr = librosa.load(file_path, sr=None)
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=top_db)
    return trimmed_audio, sr

def save_audio(audio, sr, output_path):
    """Save the processed audio to a file."""
    audio = np.clip(audio, -1.0, 1.0)
    audio = (audio * 32767).astype(np.int8)
    

    with wave.open(output_path, 'wb') as wf:
        wf.setnchannels(1)  
        wf.setsampwidth(2)  
        wf.setframerate(sr)
        wf.writeframes(audio.tobytes())

def process_and_save_files(directory_path, processing_function, output_directory, **kwargs):
    """Apply the processing function and save the results in a new directory."""
    os.makedirs(output_directory, exist_ok=True)
    wav_files = glob.glob(os.path.join(directory_path, '*.wav'))

    for wav_file in wav_files:
        try:
            processed_audio, sr = processing_function(wav_file, **kwargs)
            output_path = os.path.join(output_directory, os.path.basename(wav_file))
            save_audio(processed_audio, sr, output_path)
        except Exception as e:
            print(f"Error processing file {wav_file}: {e}")

def combined_preprocessing(file_path, bit_depth=8, top_db=15):
    audio, sr = reduce_bit_depth(file_path, bit_depth)
    audio, sr = convert_to_mono(file_path)
    audio, sr = trim_silence(file_path, top_db)
    return audio, sr

In [8]:
base_capacity = get_directory_size(audio_directory_path)
print(f"Base capacity: {base_capacity / (1024 ** 2):.2f} MB")
process_and_save_files(audio_directory_path, reduce_bit_depth, reduce_bit_depth_directory, bit_depth=4)
process_and_save_files(audio_directory_path, convert_to_mono, stereo_to_mono_directory)
process_and_save_files(audio_directory_path, trim_silence, trim_silence_directory, top_db=15)
process_and_save_files(audio_directory_path, combined_preprocessing, combined_directory, bit_depth=8, top_db=15)

bit_depth_size = get_directory_size(reduce_bit_depth_directory)
mono_size = get_directory_size(stereo_to_mono_directory)
silence_trim_size = get_directory_size(trim_silence_directory)
combined_size = get_directory_size(combined_directory)




Base capacity: 5.01 MB


## 3) Print the result for comparison:
The new results are measured to compare with original capacity for further inspection.

In [10]:
print(f"\nDirectory sizes (in MB):")
print(f"Base Directory: {base_capacity / (1024 ** 2):.2f} MB")
print(f"After Reduce Bit Depth: {bit_depth_size / (1024 ** 2):.2f} MB")
print(f"After Stereo to Mono: {mono_size / (1024 ** 2):.2f} MB")
print(f"After Trim Silence: {silence_trim_size / (1024 ** 2):.2f} MB")
print(f"After Combined Preprocessing: {combined_size / (1024 ** 2):.2f} MB")



data = {
    "Method": ["Base Directory", "Reduce Bit Depth", "Stereo to Mono", "Trim Silence", "Combined Preprocessing"],
    "Size (MB)": [
        base_capacity / (1024 ** 2),
        bit_depth_size / (1024 ** 2),
        mono_size / (1024 ** 2),
        silence_trim_size / (1024 ** 2),
        combined_size / (1024 ** 2)
    ]
}

df = pd.DataFrame(data)
print("\nDirectory Size Comparison:")
print(df)


Directory sizes (in MB):
Base Directory: 5.01 MB
After Reduce Bit Depth: 3.68 MB
After Stereo to Mono: 3.68 MB
After Trim Silence: 3.25 MB
After Combined Preprocessing: 3.25 MB

Directory Size Comparison:
                   Method  Size (MB)
0          Base Directory   5.014977
1        Reduce Bit Depth   3.683745
2          Stereo to Mono   3.683745
3            Trim Silence   3.254949
4  Combined Preprocessing   3.254949
